In [1]:
# move to project root
%cd ..
%pwd

/Users/heste/workspace/soccernet/sn-script


'/Users/heste/workspace/soccernet/sn-script'

In [2]:
import os
from dataclasses import dataclass
from pathlib import Path

import matplotlib
import numpy as np
import pandas as pd
from matplotlib import pylab as plt_lab
from matplotlib import pyplot as plt
from sn_script.config import binary_category_name
from sn_script.csv_utils import gametime_to_seconds, seconds_to_gametime
from tqdm import tqdm

# matplotlibのデフォルトフォントをTakaoGothicに設定
font = {"family": "IPAexGothic"}
matplotlib.rc("font", **font)
plt_lab.rcParams.update({"font.size": 22})

In [3]:
@dataclass
class Config:
    csv_path = Path('database/stable/scbi-v2.csv')
    sncaption_path = Path('database/misc/soccernet_labels.csv')

In [4]:
scbi_df = pd.read_csv(Config.csv_path)
sncaption_df = pd.read_csv(Config.sncaption_path)

/var/folders/8y/94zytg1j1h339lyf_x2r_ptr0000gn/T/ipykernel_89435/395221427.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  scbi_df = pd.read_csv(Config.csv_path)


In [ ]:
scbi_df.columns

In [18]:
# データの確認
target_game = "england_epl/2015-2016/2015-08-29 - 17-00 Chelsea 1 - 2 Crystal Palace"
target_game = "england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley"

scbi_df.loc[
    scbi_df['game'] == target_game,
    [ "start", "end", "text", binary_category_name]
].head(20)

,start,end,text,付加的情報か
0,0.01,2.55,"The duel has already begun, Barley is in control.",0
1,2.57,9.72,They were also measured in the first match of ...,1
2,9.78,10.86,Barley advanced.,1
3,11.30,14.47,It must be remembered that he advanced in that...,1
4,14.91,17.85,without too many difficulties.,0
5,17.87,19.22,"Exactly, that's what I was going to say.",0
6,19.24,20.20,I remember that game very well.,0
7,20.22,22.46,"Barley started well, scored a goal, but Chelse...",0
8,22.48,26.26,"That's why today shouldn't have any problems, ...",0
9,33.60,37.62,"He started with the ball to Cuadrado, left it ...",0


In [ ]:
# 映像の統計情報
# 試合数
n_games = scbi_df['game'].nunique()
print(f"Number of games: {n_games}")
# 総合時間
group_per_video = scbi_df.groupby(['game', 'half'])
durations = group_per_video['end'].max() - group_per_video['start'].min()
durations.index = group_per_video['game'].first()
total_duration = durations.sum()
print(f"Total duration: {total_duration}")


In [ ]:
# コメントの統計情報
# Total # of utterances
total_num_utterances = scbi_df.shape[0]
print(f"Total # of utterances: {total_num_utterances}")
# avg. # of utterances video (game, half)
avg_num_utterances_per_game = scbi_df.groupby(['game', 'half']).size().mean()
print(f"avg. # of utterances per game: {avg_num_utterances_per_game:.2f}")
# avg. # of characters per utterance
avg_chars_per_utterance = scbi_df['text'].dropna().apply(len).mean()
print(f"avg. # of characters per utterance: {avg_chars_per_utterance:.2f}")
# avg. length of an utterance (sec)
avg_chars_per_utterance = (scbi_df['end'] - scbi_df['start']).mean()
print(f"avg. length of an utterance: {avg_chars_per_utterance:.2f}")
# avg. length of silence (sec)
scbi_df['prev_end'] = scbi_df['end'].shift(1)
scbi_df['silence'] = (scbi_df['start'] - scbi_df['prev_end'])
scbi_df.loc[scbi_df['silence'] < 0, 'silence'] = None
silence_mean = scbi_df['silence'].mean(skipna=True)
print(f"avg. length of silence: {silence_mean:.2f}")


In [ ]:
# 分析
# 全体におけるラベルの割合
print(scbi_df[binary_category_name].value_counts(normalize=True))
# ラベルの数
print(scbi_df[binary_category_name].value_counts())

# 結果、付加的情報が18%あった
# つまり、無視できない割合存在する

In [ ]:
# 増加タイミングの分析1
# 時間帯(5分でbinning)ごとのラベルの割合
def plot_label_ratio_by_time(df: pd.DataFrame):
    fig, ax = plt.subplots(1, 2, figsize=(20, 10))

    df["start_bin"] = (
        df["start"] // 300
    )

    df_half_1 = df.loc[df["half"] == 1]
    piv_1_df = df_half_1.pivot_table(
        index="start_bin", columns=binary_category_name, aggfunc="count", values="text"
    )
    piv_1_df.index = [seconds_to_gametime(i * 300) + "~" for i in piv_1_df.index]
    (piv_1_df[1.0] / piv_1_df.sum(axis=1)).plot(
        xlabel="試合時間",
        ylabel="割合",
        title="前半",
        ylim=(0, 0.30),
        ax=ax[0]
    )
    # 後半
    half_offset = 2700
    df_half_2 = df.loc[df["half"] == 2]

    piv_2_df = df_half_2.pivot_table(
        index="start_bin", columns=binary_category_name, aggfunc="count", values="text"
    )
    piv_2_df.index = [seconds_to_gametime(i * 300 + half_offset) + "~" for i in piv_2_df.index]
    (piv_2_df[1.0] / piv_2_df.sum(axis=1)).plot(
        xlabel="試合時間",
        ylabel="割合",
        title="後半",
        ylim=(0, 0.30),
        ax=ax[1]
    )
    # 保存
    fig.savefig("database/images/scbi_v2-label_ratio_by_time.png")

plot_label_ratio_by_time(scbi_df)

In [ ]:
# 増加タイミングの分析2
# イベント周辺でのラベルの割合
def label_ratio_around_event(df: pd.DataFrame, action_df: pd.DataFrame = sncaption_df, window_size=5):
    """
    event_df (sn-captionの速報テキストデータ) の importantなコメントの前後5秒に含まれる， df の実況コメントのラベルの割合を調べる手順
    1. dfの書くコメントの is_important を False で初期化
    2. soccernet_df を game でグループ化する
    2. game ごとに，
        - soccernet_dfの各行について df の start が action_df の各アクションの start, end に含まれるかを計算する
        - 含まれるなら， df のカラム refer_$action_label を追加し，Trueにする
    3. df[refer_$action_label] = True の付加的情報かの割合を計算する
    """

    if action_df["time"].dtype == "O":
        action_df["time"] = action_df["time"].apply(gametime_to_seconds)
    action_df["start"] = action_df["time"] - 5
    action_df["end"] = action_df["time"] + 5

    # game, half, time でソート
    action_df = action_df.sort_values(by=["game", "half", "start"])

    # イベントラベルのリスト
    action_grouped = action_df.groupby("label")
    action_labels = action_grouped.size().index
    print("Unique labels:")
    print(*action_labels, sep="\n") # nanを除く

    # 各アクションのフラグ列を一時的に保持するための辞書を作成
    new_columns = {}

    # イベント周辺のコメントにフラグを立てる
    for action_label, action_df_subset in tqdm(action_grouped):
        if f"refer_{action_label}" in df.columns:
            print(f"refer_{action_label} is already in df.columns")
            continue
        new_columns[f"refer_{action_label}"] = np.zeros(len(df), dtype=bool)
        tqdm.write(f"Processing {action_label}")
        for _, action_row in tqdm(action_df_subset.iterrows()):
            # dfを直接更新するために、適切なインデックスを取得
            indices = df[
                (df["game"] == action_row["game"])
                & (df["half"] == action_row["half"])
                & (df["start"] >= action_row["start"])
                & (df["start"] <= action_row["end"])
            ].index
            new_columns[f"refer_{action_label}"][indices] = True

    # 新しい列を追加
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

    # イベント周辺のコメントの割合を計算
    for action_label in action_labels:
        if f"refer_{action_label}" not in df.columns:
            print(f"refer_{action_label} is not in df.columns")
            continue
        refer_count = df.loc[:, f"refer_{action_label}"].sum() # noqa
        additional_info_refer_count = df.loc[
            (df[f"refer_{action_label}"] == True) # noqa
            & (df[binary_category_name] == 1),
            f"refer_{action_label}",
        ].sum()


        print(f"推定の{action_label}言及コメント数: {refer_count}")
        print(f"推定の{action_label}言及付加的情報コメント数: {additional_info_refer_count}")
        if refer_count == 0:
            continue
        else:
            print(
                f"推定の{action_label}言及コメントの付加的情報の割合 {additional_info_refer_count / refer_count * 100}%"
            )

label_ratio_around_event(scbi_df, sncaption_df)

In [ ]:
# 増加タイミングの分析3
# 沈黙時間とラベルの割合の関係
def label_ratio_by_silence(df: pd.DataFrame):
    # intervalカラムを追加
    df["prev_end"] = df["end"].shift(1)
    df["interval"] = df["start"] - df["prev_end"]
    df.loc[df["interval"] < 0, "interval"] = None

    # 文字の長さを確認
    df['word_count'] = df['text'].str.split().str.len()


    # インターバルが30秒以上のものを表示
    for interval_length in [0.01, 0.1, 0.5, 1, 2, 4, 10, 20, 30, 40, 50, 60]:
        df[f"interval_over_{interval_length}"] = (
            df["interval"] > interval_length
        )
        # 全コメント　interval_length秒以上の沈黙の後の発話テキストのラベル
        rate1 = df[f"interval_over_{interval_length}"].sum() / len(df)

        # 付加的情報のみ　interval_length秒以上の沈黙の後の発話テキストのラベル
        rate2 = df[df[binary_category_name] == 1][
            f"interval_over_{interval_length}"
        ].sum() / len(df[df[binary_category_name] == 1])

        print(
            "全コメント{}秒インターバル後の発話数:{}\n付加的情報のみの{}秒インターバル後の発話数:{}".format(
                interval_length,
                df[f"interval_over_{interval_length}"].sum(),
                interval_length,
                df[df[binary_category_name] == 1][
                    f"interval_over_{interval_length}"
                ].sum(),
            )
        )

        print(f"rate1: {rate1:%}\nrate2: {rate2:%}")

        # word_countの平均
        avg_word_count = df.loc[
            df[f"interval_over_{interval_length}"] == True, # noqa
            'word_count'
        ].mean()
        print(f"word_countの平均: {avg_word_count}")


label_ratio_by_silence(scbi_df)

# scbi-v1と真逆だった最悪

In [ ]:
#データの品質チェック
# オーバーラップの確認
def overlap_check(df: pd.DataFrame):
    df['overlap'] = df["prev_end"] - df["start"]
    rate = len(df[df['overlap'] > 0]) / len(df)
    print(f"Overlapping rate: {rate:.2f}")

overlap_check(scbi_df)

In [ ]:
# ラベルの特徴
# word_countとラベルの関係 箱ひげ図で可視化
def label_by_column(column: str, df: pd.DataFrame):
    fig, ax = plt.subplots(figsize=(20, 10))
    filtered_df = df.loc[df[binary_category_name].isin([0, 1])]

    filtered_df.boxplot(column=column, by=binary_category_name, ax=ax)
    ax.set_ylim(0, 60)
    ax.set_xlabel("ラベル")
    ax.set_ylabel(column)
    ax.set_title(f"{column}とラベルの関係")
    fig.savefig(f"database/images/scbi_v2-label_by_{column}.png")

label_by_column("word_count", scbi_df)

In [ ]:
scbi_df['duration'] = scbi_df['end'] - scbi_df['start']
label_by_column("duration", scbi_df)

In [ ]:
# word_countと binary_category_nameの割合 の相関
def plot_label_ratio_by_column(column: str, df: pd.DataFrame):
    fig, ax = plt.subplots(figsize=(10, 10))
    filtered_df = df.loc[
        (df[binary_category_name].isin([0, 1])) &
        (df[column] > 0)
    ]

    # 5で割って丸めてグルーピング
    filtered_df[column] = filtered_df[column].apply(lambda x: int(x) // 5 * 5)

    # xlimを決めるために、word_countの3シグマを計算
    mu = df[column].mean()
    sigma = df[column].std()
    xlim = (max(mu - 3 * sigma, 0), round(mu + 3 * sigma))
    print(f" mu: {mu}, sigma: {sigma}")
    print(f" xlim: {xlim}")

    # xlimの範囲でデータをフィルタリング
    filtered_df = filtered_df.loc[
        (filtered_df[column] >= xlim[0]) & (filtered_df[column] <= xlim[1])
    ]


    piv_df = filtered_df.pivot_table(
        index=column, columns=binary_category_name, aggfunc="count", values="text"
    )
    piv_df = piv_df.fillna(0)
    # 割合に変換?
    piv_df = piv_df.div(piv_df.sum(axis=1), axis=0)

    piv_df.plot(
        kind="bar",
        stacked=True,
        ax=ax,
    )

    ax.set_xlabel(column)
    ax.set_ylabel("割合")
    ax.set_title(f"{column}とラベルの割合の関係")

    fig.savefig(f"database/images/scbi_v2-label_ratio_by_{column}.png")


plot_label_ratio_by_column("word_count", scbi_df)
# 文字数が多いほど付加的情報の割合が高い という当たり前のことを示した

In [ ]:
# sn-echoes avarage unique word ratio
# (unique word count divided by the the number of total words)
def avg_unique_word_ratio(df: pd.DataFrame):
    # unique_word_ratio を video (game, half) ごとに計算して平均を取る
    filtered_df = df.loc[df['text'].notnull()].copy()

    group_per_video = filtered_df.groupby(['game', 'half'])
    text_chunk = group_per_video['text'].apply(lambda x: ' '.join(x))
    unique_word_ratio = text_chunk.apply(
        lambda x: len(set(x.split())) / len(x.split())
    )

    # 平均を取る
    avg_unique_word_ratio = unique_word_ratio.mean()
    print(f"Average unique word ratio: {avg_unique_word_ratio:.3f}")
avg_unique_word_ratio(scbi_df)

# 結果、Average unique word ratio: 0.292
# sn-echoesは large-v3で0.370だった
# 低い->書き起こしの品質が悪いと、sn-echoesでは仮定していたけど、そもそもそれは間違いだと考えるべき

In [20]:
# sn-echoes にある、述語目的語の割合 円グラフを scbi-v2 に適用してみる
from collections import Counter

import nltk
import plotly.express as px
from nltk import pos_tag, word_tokenize
from nltk.corpus import (
    stopwords,  # Import the stop word list
    wordnet,
)

# 必要なデータをダウンロード
nltk.download('all')

nltk.data.path.append('/Users/heste/nltk_data')

stops = set(stopwords.words("english"))
stemmer = nltk.WordNetLemmatizer()

# 品詞タグをWordNetの形式に変換する関数
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


# 動詞-目的語ペアを抽出する関数
def extract_vn_nltk(text):
    """
    1. 文章をトークン化
    2. 品詞タグ付け
    3. stopwordを除外 3文字以上 動詞を抽出
    4. 動詞を見出し語化
    5. 動詞の後に名詞がある場合、動詞-名詞ペアを抽出
    """
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)

    vn_pairs = []
    for i, word_pos in enumerate(tagged):
        word, pos = word_pos

        #  動詞を探す stopwordを除外 3文字以上
        if pos.startswith('VB') and (word.lower() not in stops) and (len(word) >= 3):
            # 目的語となる名詞 (名詞が動詞の後にくる場合を探す)
            if i + 1 < len(tagged) and tagged[i + 1][1] in ['NN', 'NNS']:
                noun = tagged[i + 1][0].lower()
                # 動詞のみ レンマタイズ
                word = stemmer.lemmatize(word.lower(), pos=get_wordnet_pos(pos))
                vn_pairs.append((word, noun))

    return vn_pairs


def plot_vn_ratio(df: pd.DataFrame, only_bi: bool = False):
    # サンプルデータフレームのテキスト列から主語-動詞ペアを抽出
    filtered_df = df.loc[df['text'].notnull()].copy()

    if only_bi:
        # only background information
        filtered_df = filtered_df.loc[filtered_df[binary_category_name] == 1]

    # pandasのprogress_applyを使うために必要
    tqdm.pandas()

    filtered_df['vn_pairs'] = filtered_df['text'].progress_apply(extract_vn_nltk)

    print(filtered_df['vn_pairs'].str.len().describe())

    # 動詞ごとの集計
    vn_list = [
        (v, n)
        for pairs in filtered_df['vn_pairs']
        for v, n in pairs
        if len(pairs) > 0
    ]

    # 動詞の上位20個を取得
    v_list = [v for v, _ in vn_list]
    top_20_verbs = [v for v, _ in Counter(v_list).most_common(20)]

    print(f"{top_20_verbs=}")

    # 動詞ごとの目的語の上位を制限して取得
    filtered_pairs = []
    for verb in top_20_verbs:
        nouns = []
        for v, n in vn_list:
            if v == verb:
                nouns.append(n)

        noun_counts = Counter(nouns)
        print(f"{noun_counts=}")
        # 動詞の順位に応じた目的語の表示数を設定
        rank = top_20_verbs.index(verb) + 1
        if rank <= 3:
            top_nouns = noun_counts.most_common(4)  # 上位3位までの動詞には4個の名詞
        elif rank <= 5:
            top_nouns = noun_counts.most_common(3)  # 上位5位までの動詞には3個の名詞
        else:
            top_nouns = noun_counts.most_common(1)  # それ以外の動詞には1個の名詞

        print(f"{verb=}, {top_nouns=}")
        # 結果をリストに追加
        for noun, count in top_nouns:
            filtered_pairs.append((verb, noun, count))

    sunburst_df = pd.DataFrame(filtered_pairs, columns=['verb', 'noun', 'count'])

    # サンバーストチャートの作成
    fig = px.sunburst(
        sunburst_df,
        path=['verb', 'noun'],   # 動詞をメインセグメント、目的語をサブセグメントとして設定
        values='count',          # 頻度データ
        color='verb',            # 動詞ごとに色分け
        title="Verb-Noun 共起関係",
    )
    fig.show()

# plot_vn_ratio(scbi_df)

# take advantageが以上に多い
# 偏りがありそうなことがわかった


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/heste/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/heste/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/heste/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/heste/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/heste/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_dat

In [21]:
plot_vn_ratio(scbi_df, only_bi=True)

100%|██████████| 62668/62668 [00:24<00:00, 2603.19it/s]


count    62668.000000
mean         0.062520
std          0.264526
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          5.000000
Name: vn_pairs, dtype: float64
top_20_verbs=['take', 'get', 'make', 'score', 'give', 'win', 'lose', 'play', 'start', 'say', 'see', 'leave', 'put', 'create', 'attack', 'find', 'set', 'keep', 'repeat', 'change']
noun_counts=Counter({'advantage': 187, 'care': 28, 'place': 26, 'part': 14, 'charge': 12, 'something': 8, 'control': 7, 'risks': 6, 'time': 6, 'possession': 5, 'goal': 4, 'speed': 4, 'shape': 2, 'anything': 2, 'penalty': 2, 'hold': 2, 'confidence': 2, 'air': 2, 'height': 2, 'crosses': 1, 'corners': 1, 'shots': 1, 'nothing': 1, 'delaney': 1, 'office': 1, 'penalties': 1, 'exception': 1, 'sides': 1, 'chances': 1, 'numbers': 1, 'eye': 1, 'videos': 1, 'positions': 1, 'options': 1, 'refuge': 1, 'everything': 1, '5-0': 1, 'support': 1, 'knowledge': 1, 'home': 1, 'position': 1, 'momentum': 1, 'notes': 1, 'minutes':

In [5]:
# より詳細な分析のため、シーズンやチームごとにデータを分割
# game -> (ex.) england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley
# game -> league, season, date, kickoff-time, home-team, home-score, away-team, away-score
import re


def extract_data_from_game(game: str):
    league, season, match_data = game.split('/')
    # regexで抽出する
    date = re.search(r'\d{4}-\d{2}-\d{2}', match_data).group()
    kickoff_time = re.search(r'\d{2}-\d{2}', match_data).group()

    # score は 空白 数字 - 数字 空白
    home_score = int(re.search(r' (\d) - \d ', match_data).group(1))
    away_score = int(re.search(r' \d - (\d) ', match_data).group(1))

    # home-team は、 kickoff_time の後から score の前まで
    home_team = re.search(r' \d{2}-\d{2} (.*) \d - \d ', match_data).group(1)
    # away-teamは、scoreの後から終わりまで
    away_team = re.search(r' \d - \d (.*)', match_data).group(1)

    return {
        "league": league,
        "season": season,
        "date": date,
        "kickoff_time": kickoff_time,
        "home_team": home_team,
        "home_score": home_score,
        "away_score": away_score,
        "away_team": away_team,
    }
scbi_df = scbi_df.dropna(subset=['game'])
# ゲームデータを抽出
game_metadata_df = pd.DataFrame(scbi_df['game'].apply(extract_data_from_game).tolist())


In [6]:
# データの結合 # indexが同じなので、そのまま結合
scbi_df = pd.concat([scbi_df, game_metadata_df], axis=1)
scbi_df.head()

,id,game,half,start,end,text,小分類,備考,付加的情報か,language,...,src_text_y,src_text,league,season,date,kickoff_time,home_team,home_score,away_score,away_team
0,0,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,1.0,0.01,2.55,"The duel has already begun, Barley is in control.",NaN,This comment does not contain any supplementar...,0,es,...,Another opportunity for Bournemouth to get th...,"Arrancó ya el duelo, maneja la pelota el Barley.",england_epl,2014-2015,2015-02-21,15-02,Chelsea,1,1,Burnley
1,1,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,1.0,2.57,9.72,They were also measured in the first match of ...,NaN,This comment contains historical match data.,1,es,...,What a match we've had here.,Hay que decir que también se midieron en el pr...,england_epl,2014-2015,2015-02-21,15-02,Chelsea,1,1,Burnley
2,2,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,1.0,9.78,10.86,Barley advanced.,NaN,The comment contains information about the cur...,1,es,...,Incredible.,El Barley se adelantó.,england_epl,2014-2015,2015-02-21,15-02,Chelsea,1,1,Burnley
3,3,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,1.0,11.30,14.47,It must be remembered that he advanced in that...,NaN,This comment refers to a specific event in a p...,1,es,...,Just when you think the Premier League can't s...,Hay que recordar que se adelantó en ese partid...,england_epl,2014-2015,2015-02-21,15-02,Chelsea,1,1,Burnley
4,4,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,1.0,14.91,17.85,without too many difficulties.,NaN,This comment does not contain any supplementar...,0,es,...,The poacher's effort.,"Hay que recordar, sin demasiadas dificultades...",england_epl,2014-2015,2015-02-21,15-02,Chelsea,1,1,Burnley


In [7]:
scbi_df.tail()

,id,game,half,start,end,text,小分類,備考,付加的情報か,language,...,src_text_y,src_text,league,season,date,kickoff_time,home_team,home_score,away_score,away_team
338018,338030,spain_laliga/2016-2017/2017-05-21 - 21-00 Mala...,2.0,2675.31,2677.07,The ball is played by Álvaro Morata.,NaN,This comment refers to a specific event in the...,1,es,...,NaN,El balón lo pelea Álvaro Morata.,spain_laliga,2016-2017,2017-05-21,17-05,Malaga,0,2,Real Madrid
338019,338031,spain_laliga/2016-2017/2017-05-21 - 21-00 Mala...,2.0,2677.09,2679.52,"The ball goes to Málaga, who does not give up.",NaN,This comment does not contain any supplementar...,0,es,...,NaN,"La gana el Málaga, que no se rinde.",spain_laliga,2016-2017,2017-05-21,17-05,Malaga,0,2,Real Madrid
338020,338032,spain_laliga/2016-2017/2017-05-21 - 21-00 Mala...,2.0,2679.54,2680.92,"Marcelo arrives, who does not give up either.",NaN,This comment refers to a specific event in the...,0,es,...,NaN,"Llega Marcelo, que tampoco lo hace.",spain_laliga,2016-2017,2017-05-21,17-05,Malaga,0,2,Real Madrid
338021,338033,spain_laliga/2016-2017/2017-05-21 - 21-00 Mala...,2.0,2680.94,2682.06,The ball falls to Trujillo.,NaN,This comment does not contain any supplementar...,0,es,...,NaN,Le cae la pelota a Trujillo.,spain_laliga,2016-2017,2017-05-21,17-05,Malaga,0,2,Real Madrid
338022,338034,spain_laliga/2016-2017/2017-05-21 - 21-00 Mala...,2.0,2682.50,2699.59,"Sonny Kroos plays it for Cristiano, Cristiano ...",NaN,This comment refers to a specific play-by-play...,0,es,...,NaN,"Toni Kroos la juega para Cristiano, Cristiano...",spain_laliga,2016-2017,2017-05-21,17-05,Malaga,0,2,Real Madrid


In [31]:
# 言語ごとの付加的情報の割合を計算
def label_ratio_by_language(df: pd.DataFrame):
    # languageごとの付加的情報の割合を計算
    language_grouped = df.groupby('language')
    for language, group_df in language_grouped:
        print(f"言語: {language}, 試合数: {group_df['game'].nunique()}")
        print(group_df[binary_category_name].value_counts(normalize=True))
        print(group_df[binary_category_name].value_counts())

label_ratio_by_language(scbi_df)

# 結果、試合数が多い言語間での差はなさそう

言語: cy, 試合数: 1
付加的情報か
0    0.659864
1    0.340136
Name: proportion, dtype: float64
付加的情報か
0    97
1    50
Name: count, dtype: int64
言語: de, 試合数: 60
付加的情報か
0    0.795752
1    0.204226
2    0.000022
Name: proportion, dtype: float64
付加的情報か
0    35664
1     9153
2        1
Name: count, dtype: int64
言語: en, 試合数: 133
付加的情報か
0    0.804895
1    0.195078
2    0.000027
Name: proportion, dtype: float64
付加的情報か
0    89246
1    21630
2        3
Name: count, dtype: int64
言語: es, 試合数: 125
付加的情報か
 0    0.827471
 1    0.172510
-1    0.000013
 2    0.000006
Name: proportion, dtype: float64
付加的情報か
 0    130148
 1     27133
-1         2
 2         1
Name: count, dtype: int64
言語: fr, 試合数: 19
付加的情報か
0    0.809094
1    0.190906
Name: proportion, dtype: float64
付加的情報か
0    16228
1     3829
Name: count, dtype: int64
言語: it, 試合数: 2
付加的情報か
0    0.837321
1    0.161996
2    0.000684
Name: proportion, dtype: float64
付加的情報か
0    1225
1     237
2       1
Name: count, dtype: int64
言語: pl, 試合数: 1
付加的情報か
0    0.736752
1 

In [32]:
# リーグごとの付加的情報の割合を計算

def label_ratio_by_league(df: pd.DataFrame):
    # leagueごとの付加的情報の割合を計算
    league_grouped = df.groupby('league')
    for league, group_df in league_grouped:
        print(f"リーグ: {league}, 試合数: {group_df['game'].nunique()}")
        print(group_df[binary_category_name].value_counts(normalize=True))
        print(group_df[binary_category_name].value_counts())

label_ratio_by_league(scbi_df)

# england_epl, europe_uefa-champions-league　は比較的多め
# 何が言えそうかわからない

リーグ: england_epl, 試合数: 84
付加的情報か
0    0.809393
1    0.190581
2    0.000027
Name: proportion, dtype: float64
付加的情報か
0    60923
1    14345
2        2
Name: count, dtype: int64
リーグ: europe_uefa-champions-league, 試合数: 84
付加的情報か
0    0.803558
1    0.196430
2    0.000013
Name: proportion, dtype: float64
付加的情報か
0    63424
1    15504
2        1
Name: count, dtype: int64
リーグ: france_ligue-1, 試合数: 9
付加的情報か
0    0.795757
1    0.204243
Name: proportion, dtype: float64
付加的情報か
0    7165
1    1839
Name: count, dtype: int64
リーグ: germany_bundesliga, 試合数: 50
付加的情報か
0    0.796719
1    0.203255
2    0.000026
Name: proportion, dtype: float64
付加的情報か
0    30641
1     7817
2        1
Name: count, dtype: int64
リーグ: italy_serie-a, 試合数: 14
付加的情報か
0    0.830259
1    0.169666
2    0.000075
Name: proportion, dtype: float64
付加的情報か
0    11118
1     2272
2        1
Name: count, dtype: int64
リーグ: spain_laliga, 試合数: 98
付加的情報か
 0    0.830089
 1    0.169887
-1    0.000016
 2    0.000008
Name: proportion, dtype: float64
付加

In [35]:
# シーズンを4つに分割して、それぞれの付加的情報の割合を計算
from datetime import datetime


def month_to_phase(month: int):
    if month in [6, 7, 8]:
        return "シーズンオフ"
    elif month in [9, 10, 11]:
        return "序盤"
    elif month in [12, 1, 2]:
        return "中盤"
    else:
        return "終盤"


def label_ratio_by_season(df: pd.DataFrame):
    # seasonごとの付加的情報の割合を計算
    # (シーズンオフ)6~8, (序盤)9~11, (中盤)12~2, (終盤)3~5
    df["season_phase"] = df["date"].apply(lambda x: month_to_phase(datetime.strptime(x, "%Y-%m-%d").month))
    season_grouped = df.groupby('season_phase')
    for season, group_df in season_grouped:
        print(f"シーズン: {season}, 試合数: {group_df['game'].nunique()}")
        print(group_df[binary_category_name].value_counts(normalize=True))
        print(group_df[binary_category_name].value_counts())
        print()


label_ratio_by_season(scbi_df)

# 結果：
# 序盤 0.196818 -> 中盤 0.183680 -> 終盤 0.175579
# 序盤から終盤に向かって映像の説明が増える(付加的情報の割合が減る)

シーズン: シーズンオフ, 試合数: 20
付加的情報か
0    0.816222
1    0.183680
2    0.000098
Name: proportion, dtype: float64
付加的情報か
0    16695
1     3757
2        2
Name: count, dtype: int64

シーズン: 中盤, 試合数: 77
付加的情報か
0    0.81658
1    0.18342
Name: proportion, dtype: float64
付加的情報か
0    60520
1    13594
Name: count, dtype: int64

シーズン: 序盤, 試合数: 122
付加的情報か
 0    0.803149
 1    0.196818
 2    0.000025
-1    0.000008
Name: proportion, dtype: float64
付加的情報か
 0    97238
 1    23829
 2        3
-1        1
Name: count, dtype: int64

シーズン: 終盤, 試合数: 120
付加的情報か
 0    0.824405
 1    0.175579
 2    0.000008
-1    0.000008
Name: proportion, dtype: float64
付加的情報か
 0    100894
 1     21488
 2         1
-1         1
Name: count, dtype: int64



In [42]:
# 話者ごとの付加的情報の割合の差を計算
# speaker id は videoごとで別物だから
# 1. videoごとでコメントをまとめる
# 2. コメント数上位二人 の speaker idを取得
# 3. speaker ごとの付加的情報の割合を計算
# 4. speaker ごとの付加的情報の割合の差を計算

def label_ratio_by_speaker(df: pd.DataFrame):
    """話者ごとの付加的情報の割合の差を計算
    speaker id は videoごとで別物だから
    1. videoごとでコメントをまとめる
    2. コメント数上位二人 の speaker idを取得
    3. speaker ごとの付加的情報の割合を計算
    4. speaker ごとの付加的情報の割合の差を計算
    Args:
        df (pd.DataFrame): コメントデータ
    """
    # videoごとでコメントをまとめる
    group_per_video = df.groupby(['game', 'half'])
    speaker_diffs = {}
    for (game, half), video_df in group_per_video:
        # speaker id ごとの付加的情報の割合を計算
        speaker_ratios = []
        speaker_grouped = video_df.groupby('speaker')
        for speaker, speaker_df in speaker_grouped:
            comment_num = len(speaker_df)
            speaker_ratio = speaker_df[binary_category_name].value_counts(normalize=True).get(1.0, 0)
            speaker_ratios.append((speaker, comment_num, speaker_ratio))

        # コメント数上位二人 の speaker idを取得
        top_speakers = sorted(speaker_ratios, key=lambda x: x[1], reverse=True)[:2]

        # speaker idごと情報を表示
        print(f"{game=}, {half=}")
        for speaker, comment_num, speaker_ratio in top_speakers:
            print(f"{speaker=} {comment_num=} ratio:{speaker_ratio:.2f}")

        # diffを計算
        try:
            speaker_diffs[(game, half)] = abs(top_speakers[0][2] - top_speakers[1][2])
        except IndexError:
            print(f"Error: {top_speakers=}")
            pass

        print()

    # avg speaker_diffsを表示
    avg_diff = np.mean(list(speaker_diffs.values()))
    print(f"avg speaker_diffs: {avg_diff:.3f}")

label_ratio_by_speaker(scbi_df)

# 結果 avg speaker_diffs: 0.083
# 十分大きい差がありそう
# max_speaker_num=2 とか、さらにちゃんと話者分離すると、もっと大きな差が出るかもしれない

game='england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley', half=1.0
speaker='SPEAKER_01' comment_num=342 ratio:0.19
speaker='SPEAKER_00' comment_num=190 ratio:0.19

game='england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley', half=2.0
speaker='SPEAKER_00' comment_num=351 ratio:0.14
speaker='SPEAKER_02' comment_num=200 ratio:0.10

game='england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United', half=1.0
speaker='SPEAKER_01' comment_num=205 ratio:0.20
speaker='SPEAKER_04' comment_num=192 ratio:0.28

game='england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United', half=2.0
speaker='SPEAKER_02' comment_num=257 ratio:0.21
speaker='SPEAKER_01' comment_num=207 ratio:0.17

game='england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool', half=1.0
speaker='SPEAKER_01' comment_num=179 ratio:0.26
speaker='SPEAKER_00' comment_num=171 ratio:0.14

game='england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool', half=2.0
spe